In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Owner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install --upgrade gensim
!pip install -U spacy
!pip install -U pip setuptools wheel
!pip install pyldavis

In [3]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 6.1 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!pip install pyLDAvis==3.2.1

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
all = pd.read_csv('Customer Reviews Data.csv')

In [9]:
#tokenize words and clean text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(all['Statement']))

print(data_words[:1])

[['the', 'assistance', 'service', 'is', 'completely', 'useless', 'problem', 'reported', 'over', 'months', 'ago', 'by', 'email', 'and', 'phone', 'and', 'they', 'always', 'reply', 'sending', 'the', 'same', 'stupid', 'template', 'of', 'escalation', 'to', 'the', 'relevant', 'department', 'shame', 'on', 'you']]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'assistance', 'service', 'is', 'completely', 'useless', 'problem', 'reported', 'over', 'months', 'ago', 'by', 'email', 'and', 'phone', 'and', 'they', 'always', 'reply', 'sending', 'the', 'same', 'stupid', 'template', 'of', 'escalation', 'to', 'the', 'relevant', 'department', 'shame', 'on', 'you']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['assistance', 'service', 'completely', 'useless', 'problem', 'report', 'month', 'ago', 'email', 'phone', 'always', 'reply', 'send', 'stupid', 'template', 'escalation', 'relevant', 'department', 'shame']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]]


In [14]:
id2word[0]

'ago'

In [15]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ago', 1),
  ('always', 1),
  ('assistance', 1),
  ('completely', 1),
  ('department', 1),
  ('email', 1),
  ('escalation', 1),
  ('month', 1),
  ('phone', 1),
  ('problem', 1),
  ('relevant', 1),
  ('reply', 1),
  ('report', 1),
  ('send', 1),
  ('service', 1),
  ('shame', 1),
  ('stupid', 1),
  ('template', 1),
  ('useless', 1)]]

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.088*"system" + 0.085*"well" + 0.066*"mistake" + 0.040*"do" + 0.038*"euro" '
  '+ 0.036*"extra" + 0.033*"see" + 0.033*"nearly" + 0.033*"shame" + '
  '0.033*"even"'),
 (1,
  '0.045*"ask" + 0.040*"state" + 0.030*"customer" + 0.022*"incompetent" + '
  '0.022*"lose" + 0.022*"relative" + 0.022*"write" + 0.022*"due" + '
  '0.022*"electricity" + 0.022*"place"'),
 (2,
  '0.115*"bill" + 0.084*"pay" + 0.041*"charge" + 0.040*"month" + 0.032*"help" '
  '+ 0.029*"send" + 0.029*"receive" + 0.026*"arm" + 0.023*"year" + 0.023*"go"'),
 (3,
  '0.092*"work" + 0.066*"joke" + 0.059*"properly" + 0.055*"time" + '
  '0.047*"user" + 0.042*"bill" + 0.033*"personally" + 0.033*"care" + '
  '0.021*"friendly" + 0.020*"arm"'),
 (4,
  '0.087*"money" + 0.045*"unit" + 0.044*"end" + 0.044*"possible" + '
  '0.036*"consume" + 0.019*"one" + 0.019*"amount" + 0.013*"receive" + '
  '0.012*"problem" + 0.011*"tell"'),
 (5,
  '0.072*"service" + 0.070*"back" + 0.034*"thank" + 0.033*"want" + '
  '0.033*"meter" + 0.028*"wa

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.140928619932948

Coherence Score:  0.40397465862669496


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='mmds')   
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.306708  0.185949       1        1  26.591538
12     0.214777 -0.208763       2        1   7.946746
1     -0.145324 -0.236540       3        1   7.542363
10    -0.022448  0.268726       4        1   6.452123
5      0.074235 -0.223354       5        1   5.854129
7      0.239264 -0.038464       6        1   4.656711
11     0.123054  0.210484       7        1   4.418843
16    -0.196330 -0.111015       8        1   3.955135
6     -0.242633  0.024560       9        1   3.936386
18     0.102081  0.083101      10        1   3.758428
3      0.181511  0.029270      11        1   3.645065
0     -0.158745  0.213735      12        1   3.292208
15    -0.081490 -0.021364      13        1   3.131329
13    -0.195589  0.070448      14        1   3.029593
14     0.012821 -0.097976      15        1   2.830535
4     -0.047009  0.111559      16        1   2.724363
17    -0.027130 -0.206437      17        1   2.713490
9     -0.125838 -0.039144      18        1   2.175472
19    -0.004628 -0.020226      19        1   0.776516
8     -0.007286  0.005449      20        1   0.569026, topic_info=         Term       Freq      Total Category  logprob  loglift
48       bill  81.000000  81.000000  Default  30.0000  30.0000
20       help  24.000000  24.000000  Default  29.0000  29.0000
14    service  22.000000  22.000000  Default  28.0000  28.0000
92        pay  40.000000  40.000000  Default  27.0000  27.0000
326      time  22.000000  22.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
152     total   0.012804   2.344458  Topic20  -6.5975  -0.0410
367  internet   0.012804   5.335018  Topic20  -6.5975  -0.8633
377    trejqa   0.012804   1.367332  Topic20  -6.5975   0.4982
155      euro   0.012804   7.790159  Topic20  -6.5975  -1.2418
372  previous   0.012804   1.718606  Topic20  -6.5975   0.2695

[809 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
328       4  0.783803     able
328       5  0.261268     able
410      17  0.798506   access
41        1  0.859647  account
41       17  0.085965  account
...     ...       ...      ...
310       3  0.859398    write
136       1  0.851738     year
136       5  0.085174     year
112       2  0.348865      yet
112       3  0.348865      yet

[496 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 13, 2, 11, 6, 8, 12, 17, 7, 19, 4, 1, 16, 14, 15, 5, 18, 10, 20, 9])

In [20]:
pyLDAvis.save_html(vis, 'All PPTs.html')